In [1]:
import pandas as pd
import numpy as np
import duckdb
from fila_estacoes_utils import P_1 as P

ouro_conn = duckdb.connect('ouro_db')
modelling_conn = duckdb.connect('modelling_db')

# ABT Estações

In [3]:
abt_estacoes_df = ouro_conn.execute(
"""
SELECT DISTINCT
    precipitacao.id_estacao
    ,dim_estacoes.latitude
    ,dim_estacoes.longitude
    ,dim_estacoes.vl_declividade
    ,dim_estacoes.vl_altitude
    ,dim_estacoes.vl_distancia_oceano
    ,dim_estacoes.vl_aspecto_relevo
    ,precipitacao.dt_medicao
    ,precipitacao.vl_precipitacao
    ,temperatura_maxima.vl_temperatura_maxima
    ,temperatura_media.vl_temperatura_media
    ,temperatura_minima.vl_temperatura_minima
    ,umidade_maxima.vl_umidade_relativa_maxima
    ,umidade_media.vl_umidade_relativa_media
    ,umidade_minima.vl_umidade_relativa_minima
    ,vento_2m_max.vl_velocidade_vento_2m_maxima
    ,vento_2m_media.vl_velocidade_vento_2m_media
    ,vento_10m_media.vl_velocidade_vento_10m_media
FROM
    fato_estacoes_precipitacao AS precipitacao
LEFT JOIN
    fato_estacoes_temperatura_maxima AS temperatura_maxima
    ON temperatura_maxima.id_estacao = precipitacao.id_estacao
    AND temperatura_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_media AS temperatura_media
    ON temperatura_media.id_estacao = precipitacao.id_estacao
    AND temperatura_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_minima AS temperatura_minima
    ON temperatura_minima.id_estacao = precipitacao.id_estacao
    AND temperatura_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_maxima AS umidade_maxima
    ON umidade_maxima.id_estacao = precipitacao.id_estacao
    AND umidade_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_media AS umidade_media
    ON umidade_media.id_estacao = precipitacao.id_estacao
    AND umidade_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_minima AS umidade_minima
    ON umidade_minima.id_estacao = precipitacao.id_estacao
    AND umidade_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_maxima AS vento_2m_max
    ON vento_2m_max.id_estacao = precipitacao.id_estacao
    AND vento_2m_max.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_media AS vento_2m_media
    ON vento_2m_media.id_estacao = precipitacao.id_estacao
    AND vento_2m_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_10m_media AS vento_10m_media
    ON vento_10m_media.id_estacao = precipitacao.id_estacao
    AND vento_10m_media.dt_medicao = precipitacao.dt_medicao
JOIN dim_estacoes
    ON precipitacao.id_estacao = dim_estacoes.id_estacao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# ABT Produtos

In [2]:
abt_produtos_df = ouro_conn.execute(
"""
SELECT DISTINCT
   abt_estacoes.*
    ,chirps.vl_precipitacao AS vl_precipitacao_chirps
    ,cpc.vl_precipitacao AS vl_precipitacao_cpc
    ,cpc.vl_temperatura_maxima AS vl_temperatura_maxima_cpc
    ,cpc.vl_temperatura_minima AS vl_temperatura_minima_cpc
    ,gpm_final_run.vl_precipitacao AS vl_precipitacao_gpm_final_run
    ,gpm_late_run.vl_precipitacao AS vl_precipitacao_gpm_late_run
    ,power.vl_precipitacao AS vl_precipitacao_power
    ,power.vl_temperatura_maxima_2m_K AS vl_temperatura_maxima_2m_K_power
    ,power.vl_temperatura_media_2m_K AS vl_temperatura_media_2m_K_power
    ,power.vl_temperatura_minima_2m_K AS vl_temperatura_minima_2m_K_power
    ,power.vl_umidade_relativa_2m AS vl_umidade_relativa_2m_power
    ,power.vl_pressao_nivel_superficie AS vl_pressao_nivel_superficie_power
    ,power.vl_irradiancia_allsky AS vl_irradiancia_allsky_power
    ,power.vl_direcao_vento_10m AS vl_direcao_vento_10m_power
    ,power.vl_direcao_vento_2m AS vl_direcao_vento_2m_power
    ,power.vl_temperatura_orvalho_2m_K AS vl_temperatura_orvalho_2m_K_power
    ,power.vl_vento_10m AS vl_vento_10m_power
    ,power.vl_vento_medio_2m AS vl_vento_medio_2m_power
    ,power.vl_vento_maximo_2m AS vl_vento_maximo_2m_power
    ,power.vl_vento_maximo_10m AS vl_vento_maximo_10m_power
FROM abt_estacoes
JOIN fato_estacoes_latlon_produtos_df AS latlon
    ON abt_estacoes.id_estacao = latlon.id_estacao
LEFT JOIN fato_produto_chirps AS chirps
    ON latlon.lat_chirps = chirps.lat
    AND latlon.lon_chirps = chirps.lon
    AND abt_estacoes.dt_medicao = chirps.dt_medicao
LEFT JOIN fato_produto_cpc AS cpc
    ON latlon.lat_cpc = cpc.lat
    AND latlon.lon_cpc = cpc.lon
    AND abt_estacoes.dt_medicao = cpc.dt_medicao
LEFT JOIN fato_produto_gpm_final_run AS gpm_final_run
    ON latlon.lat_gpm_final_run = gpm_final_run.lat
    AND latlon.lon_gpm_final_run = gpm_final_run.lon
    AND abt_estacoes.dt_medicao = gpm_final_run.dt_medicao
LEFT JOIN fato_produto_gpm_late_run AS gpm_late_run
    ON latlon.lat_gpm_late_run = gpm_late_run.lat
    AND latlon.lon_gpm_late_run = gpm_late_run.lon
    AND abt_estacoes.dt_medicao = gpm_late_run.dt_medicao
LEFT JOIN fato_produto_power AS power
    ON latlon.lat_power = power.lat
    AND latlon.lon_power = power.lon
    AND abt_estacoes.dt_medicao = power.dt_medicao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Fila de estações de Prioridade

In [24]:
D = 30
α = 3
n_casas_decimais = 4

from fila_estacoes_utils import P_1,apply_P_as_column

fato_estacoes_base_fila_prioridade = ouro_conn.execute("SELECT * FROM fato_estacoes_base_fila_prioridade").fetch_df()
fato_estacoes_base_fila_prioridade["P_1"] = apply_P_as_column(fato_estacoes_base_fila_prioridade,P_1,D,α).round(n_casas_decimais)
fato_estacoes_base_fila_prioridade['P_1_Rank'] = fato_estacoes_base_fila_prioridade.groupby('id_estacao_base')['P_1'].rank(ascending=False,method='first').astype(int)


fato_estacoes_base_fila_prioridade = fato_estacoes_base_fila_prioridade.sort_values(by=['id_estacao_base','P_1_Rank']) \
    .reset_index(drop=True)

modelling_conn.execute("""
CREATE OR REPLACE TABLE fato_estacoes_fila_P_1 AS
SELECT * FROM fato_estacoes_base_fila_prioridade
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [27]:
fato_estacoes_base_fila_prioridade

,id_estacao_base,id_estacao_candidata,correlacao,pct_intersecao_precipitacao,vl_distancia_km,P_1,P_1_Rank
0,1,31029,0.563254,100.000000,3.00,0.5627,1
1,1,91,0.593248,100.000000,16.88,0.5035,2
2,1,927763,0.449028,95.255042,8.38,0.4186,3
3,1,927781,0.490810,100.000000,17.93,0.4045,4
4,1,927782,0.505842,100.000000,19.15,0.4014,5
...,...,...,...,...,...,...,...
19842565,1733225,926769,0.000000,0.000000,248.78,0.0000,4450
19842566,1733225,926162,0.000000,0.000000,547.39,0.0000,4451
19842567,1733225,926372,0.000000,0.000000,777.08,0.0000,4452
19842568,1733225,927897,0.000000,0.000000,492.20,0.0000,4453


# ABT Estações Vizinhas

In [ ]:
test_df = modelling_conn.execute(
"""
SELECT
    abt_produtos.*
    ,vizinha_1.correlacao AS vl_correlacao_estacao_vizinha
    ,vizinha_1.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha
    ,vizinha_1.vl_distancia_km AS vl_distancia_km_vizinha
    ,vizinha_1.P_1 AS vl_prioridade_vizinha
    ,COALESCE(COALESCE(COALESCE(COALESCE(abt_vizinha_1.vl_precipitacao,abt_vizinha_2.vl_precipitacao),abt_vizinha_3.vl_precipitacao),abt_vizinha_4.vl_precipitacao),abt_vizinha_5.vl_precipitacao) AS vl_precipitacao_vizinha
FROM abt_produtos
JOIN fato_estacoes_fila_P_1 AS vizinha_1
    ON abt_produtos.id_estacao = vizinha_1.id_estacao_base
    AND vizinha_1.P_1_Rank = 1
LEFT JOIN abt_produtos AS abt_vizinha_1
    ON vizinha_1.id_estacao_candidata = abt_vizinha_1.id_estacao
    AND abt_produtos.dt_medicao = abt_vizinha_1.dt_medicao
JOIN fato_estacoes_fila_P_1 AS vizinha_2
    ON abt_produtos.id_estacao = vizinha_2.id_estacao_base
    AND vizinha_2.P_1_Rank = 2
LEFT JOIN abt_produtos AS abt_vizinha_2
    ON vizinha_2.id_estacao_candidata = abt_vizinha_2.id_estacao
    AND abt_produtos.dt_medicao = abt_vizinha_2.dt_medicao
JOIN fato_estacoes_fila_P_1 AS vizinha_3
    ON abt_produtos.id_estacao = vizinha_3.id_estacao_base
    AND vizinha_3.P_1_Rank = 3
LEFT JOIN abt_produtos AS abt_vizinha_3
    ON vizinha_3.id_estacao_candidata = abt_vizinha_3.id_estacao
    AND abt_produtos.dt_medicao = abt_vizinha_3.dt_medicao
JOIN fato_estacoes_fila_P_1 AS vizinha_4
    ON abt_produtos.id_estacao = vizinha_4.id_estacao_base
    AND vizinha_4.P_1_Rank = 4
LEFT JOIN abt_produtos AS abt_vizinha_4
    ON vizinha_4.id_estacao_candidata = abt_vizinha_4.id_estacao
    AND abt_produtos.dt_medicao = abt_vizinha_4.dt_medicao
JOIN fato_estacoes_fila_P_1 AS vizinha_5
    ON abt_produtos.id_estacao = vizinha_5.id_estacao_base
    AND vizinha_5.P_1_Rank = 5
LEFT JOIN abt_produtos AS abt_vizinha_5
    ON vizinha_5.id_estacao_candidata = abt_vizinha_5.id_estacao
    AND abt_produtos.dt_medicao = abt_vizinha_5.dt_medicao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
len(test_df.loc[test_df['vl_precipitacao_vizinha'].isna()])/len(test_df)
# 1: 0.098
# 1,2: 0.031
# 1,2,3: 0.014
# 1,2,3,4: 0.009
# 1,2,3,4,5:

0.009031618965377566